In [1]:
# Set NEWT session id
newt_sessionid = '04d401b204b77487f4fc94cf1b92bacf'


In [2]:
# Load Gaia and connect to girder using NEWT session id
import geolib
girder = geolib.connect(girder_url='http://turtleland4:8080', newt_sessionid=newt_sessionid)


In [3]:
# Todo select item from ESS-DIVE

# As a surrogate, use file that was uploaded manually
resource_type = 'item'
resource_id = '5c88076206a1b5013e1c6d77'    # /user/admin/Public/local/SFBay_grayscale.tif
girder_url = 'girder://{}/{}'.format(resource_type, resource_id)

essdive_object = geolib.create(girder_url)


In [4]:
# Get bounds
# metadata = essdive_object.get_metadata()
metadata = {
 'bounds': {'coordinates': [[[-122.71879201711467, 37.45219874192699],
    [-122.71879201711467, 38.052231141926995],
    [-122.11875961711466, 38.052231141926995],
    [-122.11875961711466, 37.45219874192699],
    [-122.71879201711467, 37.45219874192699]]],
  'type': 'Polygon'},
 'height': 4323,
 'width': 4323}

# Create polygon to display bounds
bounds = metadata.get('bounds')
coordinates = bounds.get('coordinates')
import geojson
geometry = geojson.Polygon(coordinates)
essdive_feature = geojson.Feature(geometry=geometry, properties={'fillColor': 'black', 'fillOpacity': 0.2})
essdive_object = geolib.create(essdive_feature)

# Generate small crop geometry
bounds  = coordinates
bounds = bounds[0]
x = (bounds[0][0] + bounds[2][0]) / 2.0
y = (bounds[0][1] + bounds[2][1]) / 2.0

# Use small percentage of height & width
dx = 0.12 * (bounds[2][0] - bounds[0][0])
dy = 0.16 * (bounds[2][1] - bounds[0][1])
poly = [[
    [x,y], [x+dx,y+dy], [x-dx,y+dy], [x-dx,y-dy], [x+dx,y-dy]
]]
geometry = geojson.Polygon(poly)
crop_feature = geojson.Feature(geometry=geometry, properties={'fillColor': 'magenta', 'fillOpacity': 0.5})
crop_object = geolib.create(crop_feature)
# print(crop_object, crop_object._getdatatype())

scene = geolib.show([crop_object, essdive_object])
display(scene)


scene(center={'x': 0.0, 'y': 0.0}, layout=Layout(align_self='stretch', height='400px'))

In [5]:
import ipywidgets as widgets
z = widgets.FloatSlider(min=1, max=16, value=scene.zoom, layout=dict(width='95%'))
newlink = widgets.jslink((z, 'value'), (scene, 'zoom'))
display(z)

FloatSlider(value=9.328175, layout=Layout(width='95%'), max=16.0, min=1.0)

In [6]:
# To run on Girder:
# cropped_object = gaia.crop(essdiv_object, crop_object)
cori_job = geolib.submit_crop(essdive_object, crop_object)
cori_job

'no job id yet'